## Sample Usage IPython Notebook

In [1]:
# Install the Vinter SDK
# !pip install vintersdk

In [2]:
import pandas as pd
import os
from vintersdk import VinterAPI
from pprint import pprint

In [3]:
# Set the API key
APIKEY = os.environ.get("VINTER_API_KEY")

In [4]:
# Create instances of the VinterAPI class
vinter_multi = VinterAPI(APIKEY, "multi_assets")
vinter_single = VinterAPI(APIKEY, "single_assets")
vinter_staking = VinterAPI(APIKEY, "staking_yields")
vinter_nav = VinterAPI(APIKEY, "nav")

### Get Active Symbols

In [5]:
# Get All Active Multi-Asset Symbols Dictionary
active_symbols_multi = vinter_multi.get_all_active_data()
pprint("Active Multi-Asset Symbols")
pprint(active_symbols_multi[0])
print("-"*50)

# Get All Active Multi-Asset Symbols only
active_symbols_multi = vinter_multi.get_all_active_data(symbol_only=True)

print(f"Number of total active multi-asset symbols: {len(active_symbols_multi)}")
print(f"First 5 symbols: {active_symbols_multi[:5]}")

'Active Multi-Asset Symbols'
{'indicative_rebalance_date': None,
 'indicative_rebalance_weights': None,
 'long_name': 'Vinter Bytetree BOLD1 Inverse Volatility Index',
 'next_rebalance_date': '2023-09-29',
 'next_rebalance_weights': None,
 'next_review_date': '2023-09-22',
 'previous_rebalance_date': '2023-08-31',
 'previous_review_date': '2023-08-24',
 'short_name': 'BOLD1',
 'symbol': 'vnby-bold1-2-d',
 'weights': {'btc-usd-p-2-d': 0.24942606369464,
             'xau-usd-p-d': 0.75057393630536}}
--------------------------------------------------
Number of total active multi-asset symbols: 81
First 5 symbols: ['vnby-bold1-2-d', 'vncf-mom-15-d', 'vncf-wb3-50-d', 'vndi-ddv-30-d', 'vndi-ddv-30-h']


### Get Active Symbols for a given frequency

In [6]:
# Get All Active Multi-Asset Symbols sdk filter with frequency
# Three frequency options: "r", "d", "h"
active_symbols_multi = vinter_multi.get_all_active_data(symbol_only=True, frequency="r")

print(f"Number of filtered realtime active multi-asset symbols: {len(active_symbols_multi)}")
print(f"First 5 symbols: {active_symbols_multi[:5]}")

Number of filtered realtime active multi-asset symbols: 22
First 5 symbols: ['vndi-ddv-30-r', 'vnhx-vhashmom-12-r', 'vnk3-qualeq-30-r', 'vnpa-pando-6-r', 'vnsn-moon-10-r']


### Get Active Symbols for a given symbol

In [7]:
vntr_eq_5_d = vinter_multi.get_active_data("vntr-eq-5-d")
pprint(vntr_eq_5_d)

{'indicative_rebalance_date': None,
 'indicative_rebalance_weights': None,
 'long_name': 'Vinter Equal Weighted 5 Index',
 'next_rebalance_date': '2023-10-31',
 'next_rebalance_weights': None,
 'next_review_date': '2023-10-24',
 'previous_rebalance_date': '2023-07-31',
 'previous_review_date': '2023-07-24',
 'short_name': 'VNEQ5',
 'symbol': 'vntr-eq-5-d',
 'weights': {'ada-usd-p-d': 0.2,
             'bnb-usd-p-d': 0.2,
             'btc-usd-p-d': 0.2,
             'eth-usd-p-d': 0.2,
             'xrp-usd-p-d': 0.2}}


### Get latest data for a given symbol

In [8]:
# Get single data point
data = vinter_multi.get_latest_data(symbol="vntr-eq-5-d", limit=1)
# The data returned is in schema described in the API documentation
pprint(data[0].keys())
print("-"*50)


dict_keys(['id', 'created_at', 'symbol', 'value', 'rebalance_values', 'timestamp', 'date', 'rebalance_weights', 'current_weights', 'current_values'])
--------------------------------------------------


In [9]:
# To get 100 rows of data
data = vinter_multi.get_latest_data(symbol="vntr-eq-5-d", limit=100)
# A pandas dataframe can be created from the data
df = pd.DataFrame(data)
# Get shape of the dataframe
print(df.info())
print("-"*50)
# Get First and last 5 dates
pprint({
    "First Date": df["date"].iloc[0],
    "Last Date": df["date"].iloc[-1]
})

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 100 non-null    int64  
 1   created_at         100 non-null    object 
 2   symbol             100 non-null    object 
 3   value              100 non-null    float64
 4   rebalance_values   100 non-null    object 
 5   timestamp          100 non-null    int64  
 6   date               100 non-null    object 
 7   rebalance_weights  100 non-null    object 
 8   current_weights    100 non-null    object 
 9   current_values     100 non-null    object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.9+ KB
None
--------------------------------------------------
{'First Date': '2023-09-20', 'Last Date': '2023-06-13'}


In [10]:
# How to handle exceptions and get error messages
try:
    # Get data for a non-existent symbol
    invalia_symbol = "vntr-eq-15-d"
    data = vinter_multi.get_latest_data(symbol=invalia_symbol, limit=1)
except Exception as e:
    print(f"Exception: {e}")

Exception: {'result': 'error', 'message': 'Invalid Symbol -- Symbol does not exist', 'status_code': 400}


### Get just the latest value for a given symbol

In [11]:
# Get latest value of a btc-usd-p-d
# As it is a single asset, the instance of the VinterAPI class is different
latest_value_btc_usd = vinter_single.get_latest_value(symbol="btc-usd-p-d")
print(f"Latest BTC-USD Price: {latest_value_btc_usd} , Type: {type(latest_value_btc_usd)}")


Latest BTC-USD Price: 27155.44 , Type: <class 'float'>


### Get data for specified dates for a given symbol

In [12]:
# Get data for a single asset for a specific date (2023-01-01)
get_data_by_date_btc_usd = vinter_single.get_data_by_date(symbol="btc-usd-p-d", date="2023-01-01")
pprint(get_data_by_date_btc_usd[0].keys())

dict_keys(['id', 'created_at', 'symbol', 'value', 'timestamp', 'date'])


### Get data for Range of dates for a given symbol

In [13]:
# Get data for a single asset for a specific date (2022-01-01 to 2023-01-01)
# The data returned is in schema described in the API documentation
get_data_by_range_btc_usd = vinter_single.get_data_by_range(symbol="btc-usd-p-d", start="2022-01-01", end="2023-01-01")
df = pd.DataFrame(get_data_by_range_btc_usd)
pprint({
    "First Date": df["date"].iloc[0],
    "Last Date": df["date"].iloc[-1]
})
# The end date is not included, the last date is 2022-12-31 instead of 2023-01-01
# Due parsing of the date, the end date is set to 2023-01-01T00:00:00Z
# So the end="2023-01-01T23:59:59Z" will return the last date as 2023-01-01
# Or the end="2023-01-02" will return the last date as 2023-01-01
print("-"*50)
get_data_by_range_btc_usd = vinter_single.get_data_by_range(symbol="btc-usd-p-d", start="2022-01-01", end="2023-01-01T23:59:59Z")
df = pd.DataFrame(get_data_by_range_btc_usd)
pprint({
    "First Date": df["date"].iloc[0],
    "Last Date": df["date"].iloc[-1]
})

{'First Date': '2022-01-01', 'Last Date': '2022-12-31'}
--------------------------------------------------
{'First Date': '2022-01-01', 'Last Date': '2023-01-01'}


### Get current rebalance weight for a multi-asset symbol

In [14]:
get_multi_current_rebalance_weight_vntr_eq_5_d = vinter_multi.get_multi_current_rebalance_weight(symbol="vntr-eq-5-d")
pprint(get_multi_current_rebalance_weight_vntr_eq_5_d)
print("-"*50)

# It raises an exception if the asset type of the instance is not multi-asset
try:
    latest_value_btc_usd = vinter_single.get_multi_current_rebalance_weight(symbol="btc-usd-p-d")
except Exception as e:
    print(f"Exception: {e}")

{'ada-usd-p-d': 0.2,
 'bnb-usd-p-d': 0.2,
 'btc-usd-p-d': 0.2,
 'eth-usd-p-d': 0.2,
 'xrp-usd-p-d': 0.2}
--------------------------------------------------
Exception: The asset type must be multi_assets to use this function


### Get contribution of a single-asset symbol

In [15]:
get_single_contributions_btc_usd_p_d = vinter_single.get_single_contributions(symbol="btc-usd-p-d")
# >> ['btc-usd-p-r']

# Get contribution for the real-time btc-usd-p-r
get_single_contributions_btc_usd_p_r = vinter_single.get_single_contributions(symbol="btc-usd-p-r")
pprint(get_single_contributions_btc_usd_p_r)
print("-"*50)

# It raises an exception if the asset type of the instance is not single-asset
try:
    get_single_contributions_vntr_eq_5_d = vinter_multi.get_single_contributions(symbol="vntr_eq_5_d")
except Exception as e:
    print(f"Exception: {e}")

['btc-usd-kraken',
 'btc-usd-coinbase',
 'btc-usd-bitstamp',
 'btc-usd-itbit',
 'btc-usd-gemini',
 'btc-usd-bittrex',
 'btc-usd-gate',
 'btc-usd-cryptocom',
 'btc-usd-exmo',
 'btc-usd-ascendex',
 'btc-usd-bitfinex']
--------------------------------------------------
Exception: The asset type must be single_assets to use this function


### Get previous rebalance date for a multi-asset symbol

In [16]:
get_multi_previous_rebalance_date_vntr_eq_5_d = vinter_multi.get_multi_previous_rebalance_date(symbol="vntr-eq-5-d")
pprint(f"Previous Rebalance Date for vntr-eq-5-d: {get_multi_previous_rebalance_date_vntr_eq_5_d}")

'Previous Rebalance Date for vntr-eq-5-d: 2023-07-31'


### Get previous review date for a multi-asset symbol

In [17]:
get_multi_previous_review_date_vntr_eq_5_d = vinter_multi.get_multi_previous_review_date(symbol="vntr-eq-5-d")
pprint(f"Previous Review Date for vntr-eq-5-d: {get_multi_previous_review_date_vntr_eq_5_d}")

'Previous Review Date for vntr-eq-5-d: 2023-07-24'


### Get next review date for a multi-asset symbol

In [18]:
get_multi_next_review_date_vntr_eq_5_d = vinter_multi.get_multi_next_review_date(symbol="vntr-eq-5-d")
pprint(f"Next Review Date for vntr-eq-5-d: {get_multi_next_review_date_vntr_eq_5_d}")

'Next Review Date for vntr-eq-5-d: 2023-10-24'


### Get next rebalance date for a multi-asset symbol

In [19]:
get_multi_next_rebalance_date_vntr_eq_5_d = vinter_multi.get_multi_next_rebalance_date(symbol="vntr-eq-5-d")
pprint(f"Next Rebalance Date for vntr-eq-5-d: {get_multi_next_rebalance_date_vntr_eq_5_d}")

'Next Rebalance Date for vntr-eq-5-d: 2023-10-31'


### Get Next Rebalance Weight for a multi-asset symbol

In [20]:
get_multi_next_rebalance_weight_vntr_eq_5_d = vinter_multi.get_multi_next_rebalance_weight(symbol="vntr-eq-5-d")
pprint(f"Next Rebalance Weight for vntr-eq-5-d: {get_multi_next_rebalance_weight_vntr_eq_5_d}")

'Next Rebalance Weight for vntr-eq-5-d: None'
